<center><img src="MKn_Staffelter_Hof.jpeg" alt="Picture of old business"</center>
<!--Image Credit: Martin Kraft https://commons.wikimedia.org/wiki/File:MKn_Staffelter_Hof.jpg -->

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`businesses` and `new_businesses`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|
---
`countries`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|
---
`categories`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|

In [5]:
-- What is the oldest business on each continent?
WITH OldestBusinesses AS (
    SELECT 
        b.year_founded,
        b.business,
        c.continent,
        c.country_code
    FROM 
        businesses b
    JOIN 
        countries c ON b.country_code = c.country_code
),
RankedOldest AS (
    SELECT 
        ob.continent,
        ob.country_code,
        ob.business,
        ob.year_founded,
        ROW_NUMBER() OVER (PARTITION BY ob.continent ORDER BY ob.year_founded) as rn
    FROM 
        OldestBusinesses ob
)
SELECT 
    ro.continent,
    c.country,
    ro.business,
    ro.year_founded
FROM 
    RankedOldest ro
JOIN 
    countries c ON ro.country_code = c.country_code
WHERE 
    ro.rn = 1;

,continent,country,business,year_founded
0,Oceania,Australia,Australia Post,1809
1,Europe,Austria,St. Peter Stifts Kulinarium,803
2,Asia,Japan,Kongō Gumi,578
3,North America,Mexico,La Casa de Moneda de México,1534
4,Africa,Mauritius,Mauritius Post,1772
5,South America,Peru,Casa Nacional de Moneda,1565


In [6]:
WITH oldest_businesses AS (
    SELECT business, year_founded, category_code, country_code
    FROM businesses
    WHERE year_founded = (SELECT MIN(year_founded) FROM businesses)
    
    UNION ALL
    
    SELECT business, year_founded, category_code, country_code
    FROM new_businesses
    WHERE year_founded = (SELECT MIN(year_founded) FROM new_businesses)
),

oldest_countries AS (
    SELECT DISTINCT c.continent
    FROM oldest_businesses ob
    JOIN countries c ON ob.country_code = c.country_code
)

SELECT 
    c.continent,
    COUNT(DISTINCT c.country) AS countries_without_businesses
FROM 
    countries c
LEFT JOIN 
    businesses b ON c.country_code = b.country_code
LEFT JOIN 
    new_businesses nb ON c.country_code = nb.country_code
WHERE 
    (b.business IS NULL AND nb.business IS NULL)
GROUP BY 
    c.continent;

,continent,countries_without_businesses
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,5
4,Oceania,10
5,South America,3


In [7]:
-- Which business categories are best suited to last over the course of centuries?
WITH OldestByCategory AS (
    SELECT 
        b.category_code,
        c.continent,
        MIN(b.year_founded) as year_founded
    FROM 
        businesses b
    JOIN 
        countries c ON b.country_code = c.country_code
    GROUP BY 
        b.category_code, c.continent
)
SELECT 
    obc.continent,
    ca.category,
    obc.year_founded
FROM 
    OldestByCategory obc
JOIN 
    categories ca ON obc.category_code = ca.category_code;

,continent,category,year_founded
0,Africa,Agriculture,1947
1,North America,Agriculture,1638
2,Asia,Agriculture,1930
3,Europe,Agriculture,1218
4,Asia,Aviation & Transport,1858
5,North America,Aviation & Transport,1870
6,Africa,Aviation & Transport,1854
7,South America,Banking & Finance,1565
8,Europe,Banking & Finance,1606
9,Africa,Banking & Finance,1892
